<a href="https://colab.research.google.com/github/Siddhantg2002/Projects/blob/main/Kidney_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('data.csv')
df1 = df.copy()
df1['osmo_cond_ratio'] = df['osmo'] / df['cond']

# Create a new feature by subtracting calc from urea
df1['urea_calc_diff'] = df['urea'] - df['calc']

# Standardize the values of gravity, ph, osmo, cond, urea, and calc
df1[['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc']] = (df1[['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc']] - df1[['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc']].mean()) / df1[['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc']].std()

# Create a new feature by multiplying osmo by urea
df1['osmo_urea_interaction'] = df['osmo'] * df['urea']

# Categorize the values of gravity, ph, osmo, cond, urea, and calc into 5 bins each
df1['gravity_bin'] = pd.qcut(df['gravity'], 5, labels=False)
df1['ph_bin'] = pd.qcut(df['ph'], 5, labels=False)
df1['osmo_bin'] = pd.qcut(df['osmo'], 5, labels=False)
df1['cond_bin'] = pd.qcut(df['cond'], 5, labels=False)
df1['urea_bin'] = pd.qcut(df['urea'], 5, labels=False)
df1['calc_bin'] = pd.qcut(df['calc'], 5, labels=False)

# Load the trained model
model = load_model('my_model.h5')

# Define a function to make a prediction
def predict_target(gravity, ph, osmo, cond, urea, calc, osmo_cond_ratio, urea_calc_diff, osmo_urea_interaction, gravity_bin, ph_bin, osmo_bin, cond_bin, urea_bin, calc_bin):
    
    # Create a numpy array with the input values and reshape it for the model
    input_data = np.array([[gravity, ph, osmo, cond, urea, calc, osmo_cond_ratio, urea_calc_diff, osmo_urea_interaction, gravity_bin, ph_bin, osmo_bin, cond_bin, urea_bin, calc_bin]])
    input_data = np.reshape(input_data, (input_data.shape[0], input_data.shape[1], 1))
    # Make a prediction using the model
    prediction = model.predict(input_data)
    # Return the predicted target value
    return prediction[0][0]

#plotting
fig1 = go.Figure(data=[go.Scatter3d(
        x=df['gravity'],
        y=df['ph'],
        z=df['osmo'],
        mode='markers',
        marker=dict(
            size=5,
            color=df['target'],
            colorscale='Viridis',
            opacity=0.8
            )
    )])

fig1.update_layout(
    scene=dict(
        xaxis_title='Gravity',
        yaxis_title='pH',
        zaxis_title='Osmo'
    )
)

from pandas.plotting import scatter_matrix
scatter_matrix(df, diagonal='hist', color='b', alpha=0.3, figsize=(15,6))
fig = go.Figure(data=px.scatter_matrix(df, color='target', dimensions=['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc']))
fig.update_layout(width=800, height=800)

fig2 = px.imshow(df.corr(), color_continuous_scale='RdBu')
fig2.update_layout(
    xaxis_title='Features',
    yaxis_title='Features',
)
       
 # Add a title
st.title("Kidney Stone Predictor")

with st.sidebar:
  st.header("About")
  st.caption("Kidney stones are a common health problem that affects millions of people worldwide. Early detection and timely intervention can help prevent complications and improve treatment outcomes. However, traditional methods for diagnosing kidney stones can be invasive and time-consuming. Therefore, the aim of this project is to develop a machine learning and deep learning model that can predict the presence of kidney stones in patients based on their medical history, lab reports, and other relevant factors. The model will be trained on a large dataset of patient data and will use advanced techniques such as Convolutional Neural Networks (CNNs) and Long Short-Term Memory (LSTM) networks to achieve high accuracy. The model will be deployed on the web using Streamlit Cloud, a powerful platform for building and deploying data applications. This will allow patients and healthcare providers to easily access the model and get quick and accurate predictions about the presence of kidney stones in patients. The ultimate goal of this project is to improve the speed and accuracy of kidney stone diagnosis, leading to better patient outcomes and reduced healthcare costs.")
  st.divider()

tab1, tab2, tab3 = st.tabs(["Terminology", "Data Visiualisation", "Kiney Stone Predictor"])
with tab1:
   st.subheader('Kidney Stone Disease')
   video_file = open('myvideo.mp4', 'rb')
   video_bytes = video_file.read()
   st.video(video_bytes)
   st.divider()
   st.subheader('Urine Specific Gravity')
   st.caption("‌A urine concentration test provides the specific gravity of your urine. This measures your kidneys’ ability to balance water content and excrete waste. It is important in diagnosing some health conditions that impact water content in your urine.  urine concentration test is also called a water loading test or a water deprivation test. The specific gravity of urine refers to the electrolytes and urine osmolality. Depending on your doctor’s concerns, they give you specific instructions for eating, drinking, and taking medication prior to the test. A urine specific gravity test is used to test for diagnosing many health conditions, primarily central diabetes insipidus and nephrogenic diabetes insipidus. Both health conditions cause your body to signal excessive thirst, resulting in more urination. However, the cause of each condition is different. Damage to the pituitary gland or hypothalamus causes primary central diabetes. A malformation of your kidneys contributes to nephrogenic diabetes insipidus. Your urine’s specific gravity isn’t explicitly bad for your health. The results do signal other health conditions that may harm your health. The normal specific gravity ranges from person to person. Your urine specific gravity is generally considered normal in the ranges of 1.005 to 1.030. If you drink a lot of water, 1.001 may be normal. If you avoid drinking fluids, levels higher than 1.030 may be normal. Your doctor takes your specific symptoms, eating habits, and drinking habits into consideration when assessing your results.")
   st.subheader('pH Value')
   st.caption("A urine pH level test analyzes the acidity or alkalinity of a urine sample. It’s a simple and painless test. Many diseases, your diet, and the medications you take can affect how acidic or basic (alkaline) your urine is. For instance, results that are either too high or low can indicate the likelihood that your body will form kidney stones. If your urine is at an extreme on either the low or high end of pH levels, you can adjust your diet to reduce the likelihood of painful kidney stones. In short, your urine pH is an indicator of your overall health and gives a doctor important clues as to what’s going on in your body. In this article, we’ll go over what a normal urine pH level looks like, as well as when you need to test it, and look at the test itself. A neutral pH is 7.0. The average urine sample tests at about 6.0, but typical urine pH may range from 4.5–8.0. The higher the number, the more basic your urine is. The lower the number, the more acidic your urine is.")
   st.subheader('Osmoregulation')
   st.caption("Osmoregulation is the homeostatic control of the water potential of the blood. The kidneys are involved in filtering the blood and deciding which substances to reabsorb and which to excrete as waste. Tiny tubular structures known as tubules carry out this filtration. There are five main parts to the kidney tubules: the Bowman's capsule, the proximal convoluted tubule, the loop of Henle, the distal convoluted tubule and the collecting duct. The Bowman's capsule is used for ultrafiltration as the fluid is forced out of the blood and into the tubule. Then, the proximal convoluted tubule is used for the selective reabsorption of all glucose and some salts and water. The loop of Henle then maintains a sodium gradient, before the distal convoluted tubule makes final adjustments to the amount of water and salts that are reabsorbed. The collecting duct is the final piece of the puzzle that allows water to move out by osmosis to decrease the water content of urine.")
   st.subheader('Conductivity')
   st.caption("The urine includes urea, uric acid, protein, glucose, sodium, potassium, chlorine, inorganic phosphorus and calcium. Among the above components, uric acid, sodium, potassium, chlorine, inorganic phosphorus and calcium are electrolytes that exist in form of ions in the urine, i.e., the so-called ions of the invention. When the first electrode is electrically connected with the second electrode, positive ions in the electrical path migrate to the cathode and negative ions in the electrical path migrate to the anode to respectively generate oxidation reduction reactions. The conductivity of the urine is directly proportional to different kidney functions. More specifically, when the kidney function is normal, a filtration rate of glomeruli in the kidneys is higher, such that the quantity of these electrolytes being filtered to enter the urine is larger. Thus, the urine has a higher conductivity. Conversely, when the kidney functions is abnormal, the filtration rate of the glomeruli in the kidneys is lower, such that the quantity of these electrolytes in the urine is smaller, thus causes the urine having a lower conductivity. Therefore, by correlating the conductivity of the urine measured by the processing unit with conductivity data associated with different kidney functions, a kidney function status associated with the urine can be determined. Conductivity could be used as a parameter in routine urinalysis.")
   st.subheader('Urea Levels')
   st.caption("Urea is the principal nitrogenous waste product of metabolism and is generated from protein breakdown. It is eliminated from the body almost exclusively by the kidneys in urine, and measurement of its concentration, first in urine and later in blood, has had clinical application in the assessment of kidney (renal) function for well over 150 years. This is the first of two articles that together aim to explore current understanding of the clinical value of measuring serum or plasma urea concentration. The main focus of this first article will be physiological topics, such as urea production and renal processing of urea, along with the causes of increased and reduced plasma/serum urea concentration. Consideration will also be given to the limitations of urea measurement for assessment of renal function. The second article will deal with the value of urea measurement as an adjunct to creatinine measurement; the focus here will be the urea:creatinine ratio.")
   st.subheader('Calcium Level')
   st.caption("Our bodies can't make vitamin D. We can only get vitamin D from food and by exposing our skin to sunlight. Healthy kidneys can take that vitamin D we absorb and change it to an active form. That active vitamin D then helps us absorb calcium. But in chronic kidney disease (CKD), the kidneys are less able to make active vitamin D. Without enough active vitamin D, you absorb less calcium from the food you eat, so it then becomes low in your blood. Also, extra phosphorus in the blood of people with CKD may bind to calcium in the blood. This can then lower serum calcium. A normal serum calcium level is 8.5 - 10.2 mg/dL. A serum calcium that is either too low or too high can be dangerous and both conditions need treatment. But patients with low serum calcium, even levels at the lower end of normal, have been found to reach kidney failure faster than people with higher serum calcium levels.")
   
with tab2:
   st.subheader("Dataset")
   st.dataframe(df1.style.highlight_max(axis=0))
   st.divider()
   st.subheader("3D Scatter Plot")
   st.plotly_chart(fig1)
   st.divider()
   st.subheader('PairPlots')
   st.plotly_chart(fig)
   st.subheader('Correlation Heatmap')
   st.plotly_chart(fig2)

# Add input fields for the features
with tab3:
  gravity = st.number_input("Enter gravity value")
  ph = st.number_input("Enter pH value")
  osmo = st.number_input("Enter osmo value")
  cond = st.number_input("Enter cond value")
  urea = st.number_input("Enter urea value")
  calc = st.number_input("Enter calc value")
  osmo_cond_ratio = st.number_input("Enter osmo_cond_ratio value")
  urea_calc_diff = st.number_input("Enter urea_calc_diff value")
  osmo_urea_interaction = st.number_input("Enter osmo_urea_interaction value")
  gravity_bin = st.number_input("Enter gravity_bin value")
  ph_bin = st.number_input("Enter ph_bin value")
  osmo_bin = st.number_input("Enter osmo_bin value")
  cond_bin = st.number_input("Enter cond_bin value")
  urea_bin = st.number_input("Enter urea_bin value")
  calc_bin = st.number_input("Enter calc_bin value")
   
# Add a button to make the prediction
  if st.button("Predict"):
      prediction = predict_target(gravity, ph, osmo, cond, urea, calc, osmo_cond_ratio, urea_calc_diff, osmo_urea_interaction, gravity_bin, ph_bin, osmo_bin, cond_bin, urea_bin, calc_bin)
      st.write("Predicted Target Value: ", prediction)
      if prediction > 0.3:
          st.write("You have kidney stone.")
      else:
          st.write("You dont have kidney stone.")


Overwriting app.py


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.733s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.683s
your url is: https://cool-groups-drive-35-221-17-234.loca.lt
